### 현재 handwrite note가 audio_features랑 분리되어 있음.

- 이를 안전하게 병합할 수 있지 않을까?
- `../../datasets/dev/handwrite_notes.csv` 에 있는 fleid를 pair가 아닌 이를 먼저 개별 데이터로 전처리 해야함.
- 현재 attributes는 `anomaly_path,normal_path,anomaly_note,normal_note,common_note` 이렇게 구성되어 있다.
  - `common_note`에 있는 것은 공통점으로 `anomaly_note`와 `normal_note`에 둘다 들어가야한다. 
  - 보통 \n을 기준으로 글을 적으므로 병합할때 \n를 먼저 넣고 병합을 해야한다.
- 그렇기 이를 `path,note` 이렇게 분리해야한다.
- 병합된 파일은 `../../datasets/dev/handwrite_notes_single.csv` 로 한다.
- 먼저 이렇게 single csv를 만든뒤 `audio_features.json`에 바로 넣지 말고 `audio_features_merged.json`으로 파일을 복사하여 여기에 병합한다.

In [7]:
import pandas as pd
import os

def preprocess_handwrite_notes(input_csv_path, output_csv_path):
    # CSV 파일 읽기
    df = pd.read_csv(input_csv_path)
    
    # 결과를 저장할 리스트 초기화
    records = []
    
    # 각 행에 대해 처리
    for idx, row in df.iterrows():
        # 공통 노트 처리
        common_note = row['common_note'] if not pd.isnull(row['common_note']) else ''
        common_note = common_note.strip() if common_note else ''
        
        # Anomaly 데이터 처리
        anomaly_path = row['anomaly_path']
        anomaly_note = row['anomaly_note'] if not pd.isnull(row['anomaly_note']) else ''
        anomaly_note = anomaly_note.strip() if anomaly_note else ''
        combined_anomaly_note = '\n'.join(filter(None, [common_note, anomaly_note]))
        records.append({'path': anomaly_path, 'note': combined_anomaly_note})
        
        # Normal 데이터 처리
        normal_path = row['normal_path']
        normal_note = row['normal_note'] if not pd.isnull(row['normal_note']) else ''
        normal_note = normal_note.strip() if normal_note else ''
        combined_normal_note = '\n'.join(filter(None, [common_note, normal_note]))
        records.append({'path': normal_path, 'note': combined_normal_note})
    
    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame(records)
    
    # CSV로 저장
    result_df.to_csv(output_csv_path, index=False)

if __name__ == "__main__":
    input_csv = '../../datasets/dev/handwrite_notes.csv'
    output_csv = '../../datasets/dev/handwrite_notes_single.csv'
    preprocess_handwrite_notes(input_csv, output_csv)
    print(f"'{output_csv}' 파일이 생성되었습니다.")


'../../datasets/dev/handwrite_notes_single.csv' 파일이 생성되었습니다.


# 병합하기 위해선 밑의 코드를 실행하면 된다!

In [ ]:
import json
import pandas as pd
import os

def merge_notes_into_audio_features(audio_features_json_path, notes_csv_path, output_json_path, datasets_dir):
    """
    Merge handwrite notes into audio features JSON.

    Parameters:
        audio_features_json_path (str): Path to the audio_features.json file.
        notes_csv_path (str): Path to the processed handwrite_notes_single.csv file.
        output_json_path (str): Path to save the merged JSON file.
        datasets_dir (str): Base directory of the datasets.
    """
    # audio_features.json 로드
    with open(audio_features_json_path, 'r', encoding='utf-8') as f:
        audio_features_data = json.load(f)
    
    # handwrite_notes_single.csv 로드
    notes_df = pd.read_csv(notes_csv_path)
    
    # 'path'와 'note'를 딕셔너리로 변환 (전체 경로 기준)
    # 'note' 필드가 NaN인 경우 빈 문자열로 대체
    notes_df['note'] = notes_df['note'].fillna('')
    notes_dict = dict(zip(notes_df['path'], notes_df['note']))
    
    # 오디오 특징 데이터에 노트 병합
    for entry in audio_features_data['audio_features']:
        file_path = entry['file_path']
        
        # `file_path`를 `datasets_dir` 기준으로 전체 경로로 변환
        # 예: "../unziped/dev/ToyCar/test/file.wav" -> "../../../datasets/dev/ToyCar/test/file.wav"
        if file_path.startswith('../unziped/dev/'):
            relative_path = file_path.replace('../unziped/dev/', '')
            normalized_path = os.path.join('../../../datasets/dev', relative_path)
        else:
            # 다른 경우, 필요한 경우 추가 처리
            normalized_path = file_path  # 기본값으로 설정
        
        # 'note' 필드가 문자열인지 확인하고, 아니면 빈 문자열로 설정
        handwritten_note = notes_dict.get(normalized_path, '')
        if not isinstance(handwritten_note, str):
            handwritten_note = ''
        else:
            handwritten_note = handwritten_note.strip()
        
        # 'note' 필드 업데이트
        if not handwritten_note:
            entry['note'] = ''
        else:
            entry['note'] = handwritten_note
    
    # 병합된 데이터 저장 (ensure_ascii=False로 설정하여 한글이 유니코드로 저장되지 않도록 함)
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(audio_features_data, f, indent=4, ensure_ascii=False)
    
    print(f"병합된 JSON 파일이 '{output_json_path}' 경로에 저장되었습니다.")

if __name__ == "__main__":
    audio_features_json = 'audio_features.json'
    notes_csv = '../../datasets/dev/handwrite_notes_single.csv'
    output_json = 'audio_features_merged.json'
    datasets_dir = "../../../datasets/dev"  # 실제 데이터셋 경로로 수정하세요
    merge_notes_into_audio_features(audio_features_json, notes_csv, output_json, datasets_dir)
